In [ ]:
import requests 
import pandas as pd
from requests.auth import HTTPBasicAuth 
import io
import numpy as np

In [ ]:
API_key = " " 

## 1) Download the original file from Alphacast

In [ ]:
#Define the url, the id and the extension of the dataset
url = 'https://www.alphacast.io/api/datasets/7337/data.csv?&api_key='

df = pd.read_csv(url, sep = ',')

df = df.set_index('Year')
df

## 2) Transformation: Hodrick Prescott filter

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
#Create a dataframe to save the transformed info
df_hp = pd.DataFrame()

#Apply the HP filter on each of the columns of the dataframe
for col in df.columns[1:]:
    target = df[col]
    cycle, trend = sm.tsa.filters.hpfilter(target, 129600)
    
    #Store the results
    df_hp[col] = df[col]
    df_hp[col+ '- Cycle'] = cycle
    df_hp[col+ '- Trend'] = trend
    
    #Plot the series
    fig, ax = plt.subplots()
    df_hp[[col, col+ '- Trend']]['1992-01-01':].plot(ax=ax, fontsize=7)
    plt.show()

In [ ]:
#Print the transformed dataframe
pd.set_option('display.max_columns', None)
df_hp

In [ ]:
#Rename the index to upload the dataframe
df_hp = df_hp.rename_axis('Date')

## 3) Upload transformed data to Alphacast

### 3.1) Create a repository

In [ ]:
#Define the url for the request, and the parameters of the repo
url = "https://api.alphacast.io/repositories"

form = {
   "name": "Webinar Test Repo",
   "description": "This is a test repository for the Webinar",
   "privacy": "Private", 
   "slug": "web-test-repo"
}

r = requests.post(url, data=form, auth=HTTPBasicAuth(API_key, ""))

#To print the details of the new repo
pd.DataFrame(eval(r.content.decode('utf-8')), index=[0])

### 3.2) Create a dataset

In [ ]:
#Define the url for the request, and the parameters of the dataset
url = "https://api.alphacast.io/datasets"

form = {
   "name": "USA Trade Balance - HP",
   "repositoryId": ,
    "description": "USA 1992-2020 trade balance series with Hodrick-Prescott filter applied"
}

r = requests.post(url, auth=HTTPBasicAuth(API_key, ""), data=form)

#Print the details of the new dataset
pd.DataFrame(eval(r.content.decode('utf-8')), index=[0])

### 3.3) Uploading data 

The API to upload data to a dataset requires two parameters:

* **deleteMissingFromDB**: True / False. This parameter indicates, if True, that if the data that was in the dataset is not in the new data being uploaded, it should be deleted from the database.
* **onConflictUpdateDB**: True / False. This parameter indicates, if True, that if there is a conflict between the data that is already in the database and the new ones, it keeps the new data and overwrites the previous ones.

In [ ]:
#Define the url for the request, and the parameters for uploading the data
url_base = 'https://api.alphacast.io/datasets/'
dataset_id = ''
additional_params = '/data?deleteMissingFromDB=True&onConflictUpdateDB=True'

url = url_base + dataset_id + additional_params

files = {'data': df_hp.to_csv()}

r = requests.put(url, files=files, auth=HTTPBasicAuth(API_key, ""))

pd.DataFrame(eval(r.content.decode('utf-8')), index=[0])